In [1]:
import sys

sys.path.append('/home/anton/Documents/Tulane/Research/demo-vae/pip/src/')

from demovae import DemoVAE

field = 'f_4079_0_0'

vae = DemoVAE(pperiod=50, use_cuda=True, nepochs=10000, latent_dim=120, weight_decay=0, lr=5e-5, loss_rec_mult=1e3,
              loss_pred_mult=0.01, loss_C_mult=1e4)
vae.load(f'/home/anton/Documents/Yale/UKB/VAE_Models/700_subs/{field}.pkl')

print('Done')

/home/anton/Documents/Tulane/Research/demo-vae/pip/src/demovae/sklearn.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dct = torch.load(path)


Done


In [2]:
import pickle

dct = pickle.load(open(f'/home/anton/Documents/Yale/UKB/VAE_Models/700_subs/demos_{field}.pkl', 'rb'))

print(list(dct.keys()))
print('Done')

['demos', 'types']
Done


In [8]:
# Load ADNI model

import pickle

adni = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/ADNI/weightsbalanced.pkl', 'rb'))

list(adni.keys())

['intercept', 'coef']

In [9]:
dct['demos'][3][:20]

array([164., 176., 179., 165., 157., 169., 166., 170., 173., 178., 178.,
       164., 182., 175., 160., 161., 176., 154., 166., 187.])

In [10]:
import numpy as np

vals = []

for i in range(20):
    synth = vae.transform(len(dct['demos'][0]), dct['demos'], dct['types'])
    
    b = adni['intercept']
    w = adni['coef']
    sum_logits = 0
    
    for fc in synth:
        logits = b+np.sum(fc*w)
        sum_logits += logits
    
    vals.append(sum_logits/len(synth))

print(np.mean(vals))
print(np.std(vals))

-18.339021612351758
0.33032462743009766


In [13]:
vals = []

for i in range(20):
    ndemo = []
    for d in dct['demos']:
        dd = d+0
        ndemo.append(dd)

    # ndemo[1] = 1*np.ones(len(ndemo[1]))
    # ndemo[3] += 30
    # ndemo[4] = 1*np.ones(len(ndemo[4]))
    ndemo[3] -= 30
    
    nsynth = vae.transform(700, ndemo, dct['types'])
    
    sum_logits = 0
    
    for fc in nsynth:
        logits = b+np.sum(fc*w)
        sum_logits += logits
    
    vals.append(sum_logits/len(nsynth))

print(np.mean(vals))
print(np.std(vals))

-15.686610636967174
0.3822017266898549
